# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("homepage_actions.csv")

# Basic exploration
print("Shape of data:", df.shape)
print(df.head())
print(df.info())

# Unique actions
print("\nUnique actions:", df['action'].unique())

# Check overlap of IDs
print("\nTotal unique users:", df['id'].nunique())
print("Total rows:", df.shape[0])

# Count by group and action
group_action_counts = df.groupby(['group', 'action'])['id'].nunique().unstack(fill_value=0)
print("\nCounts by group and action:\n", group_action_counts)

# Did anyone click who didn’t view?
users_viewed = set(df[df['action']=="view"]["id"])
users_clicked = set(df[df['action']=="click"]["id"])
print("\nUsers who clicked but did not view:", len(users_clicked - users_viewed))

# Check overlap between control and experiment groups
users_control = set(df[df['group']=="control"]["id"])
users_experiment = set(df[df['group']=="experiment"]["id"])
print("\nOverlap between groups:", len(users_control & users_experiment))


Shape of data: (8188, 4)
                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None

Unique actions: ['view' 'click']

Total unique users: 6328
Total rows: 8188

Counts by group and action:
 action      click  view
group                  
control       932  3332
experiment    928  2996

Users who clicked but did not vi

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Create binary variable: 1 if clicked, 0 if not
df_binary = df.pivot_table(index="id", columns="action", values="group", aggfunc="first")

# Replace NaN with 0 for clicks
df_binary['clicked'] = df_binary['click'].notnull().astype(int)

# Map each user to their group
df_binary['group'] = df_binary['view']

# Calculate clicks by group
clicks = df_binary.groupby("group")['clicked'].sum()
views = df_binary.groupby("group")['clicked'].count()

# Z-test
stat, pval = proportions_ztest([clicks['experiment'], clicks['control']],
                               [views['experiment'], views['control']])

print("Z-statistic:", stat)
print("P-value:", pval)


Z-statistic: 2.618563885349469
P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
#Your code here
p_control = clicks['control'] / views['control']
expected_clicks_exp = views['experiment'] * p_control
print("Expected clicks in experiment group:", expected_clicks_exp)
print("Actual clicks in experiment group:", clicks['experiment'])


Expected clicks in experiment group: 838.0168067226891
Actual clicks in experiment group: 928


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [4]:
#Your code here
# Variance formula: n * p * (1-p)
variance = views['experiment'] * p_control * (1 - p_control)
std_dev = variance ** 0.5

z_score = (clicks['experiment'] - expected_clicks_exp) / std_dev
print("Z-score:", z_score)


Z-score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [5]:
#Your code here
from scipy.stats import norm

p_value_manual = 1 - norm.cdf(z_score)   # one-tailed test
print("Manual P-value:", p_value_manual)


Manual P-value: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: 
**
Yes — the results roughly match:

Both methods show that the experiment group had significantly more clicks than expected under the null hypothesis.

Both p-values are well below 0.05, so we reject the null hypothesis.

This means the experimental homepage was more effective at driving clicks than the control.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.